In [175]:
import requests
import pandas as pd

def extract_job_data() -> pd.DataFrame:
    url = 'https://api.hh.ru/vacancies'
    params = {
        'area': [1, 2, 53, 66],              # Москва, Санкт-Петербург, Краснодар, Нижний Новгород
        'ored_clusters': True,
        'text': 'data',
        'professional_role': [96, 156, 165], # Разработчик (программист), Data Scientist, Data Analyst
        'search_period': 1,
        'per_page': 20
    }

    with requests.Session() as session:
        req = session.get(url, params=params)
        req.raise_for_status()
        raw_data = req.json()

        items = raw_data['items']
        for page in range(1, raw_data['pages']):
            params['page'] = page
            req = session.get(url, params=params)
            req.raise_for_status()
            data = req.json()
            items.extend(data['items'])

        df = pd.json_normalize(items)

        # Фильтрация по профессии
        professions = ['Инженер данных', 'Инженер по данным', 'Дата инженер', 'Data Engineer', 'Разработчик ETL', 'Разработчик DWH', 'Разработчик хранилищ данных']
        df = df[df['name'].str.contains('|'.join(professions), case=False)]

        return df

In [ ]:
def transform_job_data() -> pd.DataFrame:
    # Используем метод `extract_job_data()` только один раз
    df = extract_job_data()

    # Выбираем только необходимые столбцы с помощью списка
    columns_to_select = [
        'id', 'name', 'published_at', 'created_at', 'alternate_url', 'accept_temporary',
        'area.id', 'area.name', 'salary.from', 'salary.to', 'salary.currency', 'salary.gross',
        'employer.id', 'employer.name', 'employer.alternate_url', 'employer.accredited_it_employer',
        'employer.trusted', 'snippet.requirement', 'snippet.responsibility', 'schedule.name',
        'experience.name', 'employment.name', 'department.name', 'address.raw'
    ]
    df_filtered = df[columns_to_select]

    # Заменяем точки на подчеркивания в названиях столбцов
    df_filtered.columns = df_filtered.columns.str.replace('.', '_', regex=False)

    return df_filtered

transform_job_data()